# Machine Learning (Summer 2017)

## Homework 2:


- Implement Naive Bayes model (remember about smoothing). 
- Find a reasonably interesting but not to complicated dataset for which you will be able to use this model to perform binary classification. Do the latter.
- Produce the confussion matrix, calculate accuracy, precission, recall
- Check how your model does against its version from sklearn and logistic regression from sklearn. 

In [8]:
# https://en.wikipedia.org/wiki/Conditional_probability#Kolmogorov_definition
def conditional_probability(joint_a_b: float, b: float):
    if not 0 < b:
        return None
    return joint_a_b / b

In [10]:
# https://en.wikipedia.org/wiki/Bayes%27_theorem#Statement_of_theorem
def bayes_equation(a: float, b: float, joint_a_b: float):
    if not b:
        return None
    return (conditional_probability(joint_a_b, a) * a) / b